# Исследование рынка общественного питания Москвы

Заказчик этого исследования - фонд «Shut Up and Take My Money».

Нужно подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего места для заведения общественного питания.

Нам доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

### Описание данных

Файл `moscow_places.csv`:

- `name` — название заведения;
- `address` — адрес заведения;
- `category` — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- `hours` — информация о днях и часах работы;
- `lat` — широта географической точки, в которой находится заведение;
- `lng` — долгота географической точки, в которой находится заведение;
- `rating` — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — `5.0`);
- `price` — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- `avg_bill` — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:
    - «Средний счёт: 1000–1500 ₽»;
    - «Цена чашки капучино: 130–220 ₽»;
    - «Цена бокала пива: 400–600 ₽».
        
        и так далее;
        
- `middle_avg_bill` — число с оценкой среднего чека, которое указано только для значений из столбца `avg_bill`, начинающихся с подстроки «Средний счёт»:
    - Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
    - Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
    - Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.
- `middle_coffee_cup` — число с оценкой одной чашки капучино, которое указано только для значений из столбца `avg_bill`, начинающихся с подстроки «Цена одной чашки капучино»:
    - Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
    - Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
    - Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.
- `chain` — число, выраженное `0` или `1`, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):
    - 0 — заведение не является сетевым
    - 1 — заведение является сетевым
- `district` — административный район, в котором находится заведение, например Центральный административный округ;
- `seats` — количество посадочных мест.

### План работ

**Шаг 1.** **Изучим файлы с данными**
1. Загрузим данные из файлов в датафрейм.
2. Изучим общую информацию о полученном датафрейме.

**Шаг 2. Предобработка данных**

1. Найдём и изучим пропущенные значения в столбцах
2. Рассмотрим типы данных в каждом столбце
3. Изучим уникальные значения и дубликаты

**Шаг 3. Проведём исследовательский анализ данных:**

1. Изучим параметры заведений
2. Посмотрим, какие факторы больше всего влияют на целевые задачи исследования

**Шаг 4. Исследование по открытию кофейни**

**Шаг 5. Сделаем выводы и дадим рекомендации (презентация)**

### Оглавление

1. [Загрузка данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка дубликатов](#duplicates)
    * [Обработка пропущенных значений](#null)
    * [Типы данных](#dtypes)
    * [Дополнительные столбцы](#newcolumns)
3. [Анализ данных](#eda)
4. [Открытие кофейни](#coffeeshop)
5. [Подготовка презентации](#presentation)

<a id="start"></a>
## Загрузка данных

In [ ]:
# импорт библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# импортируем карту, маркер и хороплет
from folium import Map, Marker, Choropleth
# импортируем кластер
from folium.plugins import MarkerCluster

# зададим стиль по-умолчанию для графиков
plt.style.use('ggplot')

In [ ]:
# загружаем датасет из файла
try:
    data = pd.read_csv('/datasets/moscow_places.csv')
except:
    data = pd.read_csv('../datasets/moscow_places.csv')

In [ ]:
# выведем первые 5 строк датасета для ознакомления
data.head()

In [ ]:
# общая информация о датафрейме
data.info()

### Промежуточные итоги

В датасете представлено 8406 заведений.
- `name` — название заведения задано строками (стоит проверить на дубликаты);
- `category` — категория заведения, задано строками, стоит проверить на неявные дубликаты и уникальные значения;
- `address` — адрес заведения, задано строками (стоит проверить на дубликаты вместе с названием);
- `district` — административный район, в котором находится заведение, задано строками;
- `hours` — информация о днях и часах работы, задано строками, может быть перечислением, есть пропуски
- `lat` — широта географической точки, в которой находится заведение - вещественное значение;
- `lng` — долгота географической точки, в которой находится заведение - вещественное значение;
- `rating` — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — `5.0`) - вещественное значение;
- `price` — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее - задано строками, стоит проверить уникальные значения, много пропусков (отсутствует информация - пропуски можно заменить заглушкой);
- `avg_bill` — строка, которая хранит среднюю стоимость заказа в виде диапазона, - задано строками, много пропусков (отсутствует информация - пропуски можно заменить заглушкой);
- `middle_avg_bill` — число с оценкой среднего чека, вещественное, много пропусков
- `middle_coffee_cup` — число с оценкой одной чашки капучино, вещественное, много пропусков, но должна быть сумма с предыдущим столбцом
- `chain` — число, выраженное `0` или `1`, целочисленное (можно заменить на boolean)
- `seats` — количество посадочных мест, много пропусков (отсутствует информация), можно заменить на целочисленные

<a id="preprocessing"></a>
## Предобработка данных

<a id="duplicates"></a>
### Обработка дубликатов

In [ ]:
# создадим отдельную переменную для очищенных данных
df = data.copy()

In [ ]:
# ищем полные дубликаты
df.duplicated().sum()

In [ ]:
# дубликаты по названию
df.duplicated(subset=['name']).sum()

In [ ]:
df[df.duplicated(subset=['name'])].head(10)

Дубликатов по названию очень много, так как есть сети и неспецифические названия вроде "Кафе". Попробуем посмотреть название вместе с адресом

In [ ]:
# дубликаты по названию и расположению
df.duplicated(subset=['name', 'address']).sum()

In [ ]:
# Проверим дубликаты, предварительно приведя значения к нижнему регистру
lowercase = df.copy()
lowercase['name'] = lowercase['name'].str.lower()
lowercase['address'] = lowercase['address'].str.lower()

lowercase.duplicated(subset=['name', 'address']).sum()

In [ ]:
# посмотрим на найденные
lowercase[lowercase.duplicated(subset=['name', 'address'], keep=False)]

In [ ]:
# найдём эти заведения в оригинальном датасете
df[lowercase.duplicated(subset=['name', 'address'], keep=False)]

Неявных дубликатов немного. Можно было бы создать временные столбцы с нижним регистром для `drop_duplicates`, но тут, кажется, можно удалить дубликаты вручную. (проверили их по признаку сетевых, удаляем соответствующие)

In [ ]:
df = df.drop([215, 1430, 2211, 3091])

<div class="alert alert-info">
Денис: нашлось 4 дубликата. Удалил вручную, lowercase в адресе тоже надо учесть там, где делается анализ по улицам
    
 
</div>

In [ ]:
# уникальные значения в категориях
df['category'].value_counts()

In [ ]:
# уникальные значения в категориях
df['district'].value_counts()

In [ ]:
# уникальные значения в категориях
df['price'].value_counts()

In [ ]:
# уникальные значения в категориях
df['chain'].value_counts()

<a id="null"></a>
### Обработка пропущенных значений

In [ ]:
# количество пропусков в столбцах
df.isna().sum()


- `hours` — пропуски скорее всего из-за отсутствия информации, заменим на `'unknown'`
- `price` — отсутствует информация - пропуски можно заменить заглушкой
- `avg_bill` — отсутствует информация - пропуски можно заменить заглушкой
- `middle_avg_bill` — оставим пропуски, данные зависят от содержимого `avg_bill`
- `middle_coffee_cup` — оставим пропуски, данные зависят от содержимого `avg_bill`
- `seats` — оставим пропуски, так как нет информации, а значения численные (0 будет неправильным)

In [ ]:
# заменим пустые значения на заглушки
df['hours'] = df['hours'].fillna('unknown')
df['price'] = df['price'].fillna('unknown')
df['avg_bill'] = df['avg_bill'].fillna('unknown')

In [ ]:
# количество пропусков в столбцах
df.isna().sum()

<a id="newcolumns"></a>
### Дополнительные столбцы

In [ ]:
df.head()

- Создадим столбец `street` с названиями улиц из столбца с адресом.

In [ ]:
# формат адреса
df['address'].unique()

In [ ]:
def second_value(string):
    """
    Возвращает вторые значения из списка, который передаётся строкой
    """
    
    try:
        list = string.split(',')
        return list[1].strip()
    except:
        return 'error'

In [ ]:
df['street'] = df['address'].apply(second_value)

In [ ]:
# посмотрим что получилось
df['street'].unique()

- Создадим столбец `is_24/7` с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
    - логическое значение `True` — если заведение работает ежедневно и круглосуточно;
    - логическое значение `False` — в противоположном случае.

In [ ]:
df['is_24/7'] = df['hours'].str.contains('ежедневно, круглосуточно', regex=False)

In [ ]:
# посмотрим что получилось
df['is_24/7'].value_counts()

### Промежуточные итоги
- значимых дубликатов в данных не замечено
- в трёх столбцах пропуски связаны с недостатком данных, заменили их на значение `'unknown'`, ещё в трёх столбцах пропуски оставили
- создали новые столбцы

<a id="eda"></a>
## Анализ данных

- Какие категории заведений представлены в данных? Исследуем количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. 

In [ ]:
# группируем данные по категориям6 считаем количество заведений
categories = df.groupby('category').agg(total=('name','count'))
categories

Построим визуализацию

In [ ]:
# сортируем по количеству
categories.sort_values(by='total', inplace=True, ascending=True)

# размер графика
fig, ax = plt.subplots(figsize=(10,6), facecolor=(.94, .94, .94))

ax.barh(categories.index, categories['total'])

# аннотации
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')
 
# заголовок
ax.set_title('Распределение заведений по категориям',
             loc ='left', )

ax.set_xlabel("Количество заведений")
ax.set_ylabel("Категория заведения")

plt.show()

**Вывод**
Больше всего в данных представлены кафе, рестораны и кофейни. 
Кофейни на третьем месте. Это означает высокую конкуренцию и необходимость грамотно подходить к выбору места для открытия.

- Исследуем количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. 

In [ ]:
plt.figure(figsize=(12, 6)) 

sns.stripplot(x='category', y='seats', data=df)
plt.xlabel('категории заведений')
plt.ylabel('количество посадочных мест')
plt.title('Количество посадочных мест по категориям')
plt.show()

Очень мало заведений с количеством посадочных мест > 400

In [ ]:
# 
plt.figure(figsize=(12, 5)) 
ax = sns.boxplot(x='category', y='seats', data=df)
plt.ylim(top=400)
plt.xlabel('категории заведений')
plt.ylabel('количество посадочных мест')
plt.title('Количество посадочных мест по категориям')
plt.show()

**Вывод**
Большинство заведений имеют менее 300 посадочных мест. Большая часть заведений укладывается в диапазон 50-150 посадочных мест

- Рассмотрим и изобразим соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?


In [ ]:
chains = df.groupby('chain').agg(total=('name','count'))
chains.head()

In [ ]:
ax = plt.bar(
    height=chains['total'], 
    x=['несетевые', 'сетевые']
    )

# Add Title 
plt.title(
    label="соотношение сетевых и несетевых заведений", 
    fontdict={"fontsize":16},
    pad=20
)

plt.show()

**Вывод**
В датасете больше несетевых заведений. 

- Какие категории заведений чаще являются сетевыми?


In [ ]:
# создадим сводную таблицу по признаку сетевых заведений
chain_categories = df.pivot_table(index='category', columns='chain', values='address', aggfunc='count')
chain_categories

In [ ]:
# данные
groups = chain_categories.index
values1 = chain_categories[0]
values2 = chain_categories[1]

# размер графика
fig, ax = plt.subplots(figsize=(12,6), facecolor=(.94, .94, .94))

# Stacked bar chart
ax.bar(groups, values1, label = "Несетевые")
ax.bar(groups, values2, bottom = values1, label = "Сетевые")

# Labels
for bar in ax.patches:
  ax.text(bar.get_x() + bar.get_width() / 2,
          bar.get_height() / 2 + bar.get_y(),
          round(bar.get_height()), ha = 'center',
          color = 'w', weight = 'bold', size = 10)

ax.legend()
ax.set_ylabel('Количество заведений')
ax.set_xlabel('Категория заведения')

plt.title('Количество сетевых и несетевых заведений')

plt.show()

**Вывод**
Чаще всего сетевыми являются булочные и пиццерии. Кофейни тоже чаще сетевые, чем несетевые.

In [ ]:
# создадим новую сводную таблицу по признаку сетевых заведений
chain_categories_2 = df.groupby('category').agg(total = ('name', 'count'), chain = ('chain', 'sum'))
# добавим %
chain_categories_2['chain_pct'] = round(chain_categories_2['chain']/chain_categories_2['total']*100)
chain_categories_2 = chain_categories_2.sort_values(by = 'chain_pct', ascending=False)
chain_categories_2

In [ ]:
# размер графика
fig, ax = plt.subplots(figsize=(12,5), facecolor=(.94, .94, .94))

ax.bar(chain_categories_2.index, chain_categories_2['chain_pct'])
 
# заголовок
ax.set_title('% сетевых заведений по категориям',
             loc ='left', )

ax.set_ylabel("% сетевых в общем количестве заведений")
ax.set_xlabel("Категория заведения")

plt.show()

- Сгруппируем данные по названиям заведений и найдём топ-15 популярных сетей в Москве. Под популярностью понимается количество заведений этой сети в регионе. 


In [ ]:
# подготовим данные
top_chains = df.groupby('name').agg(total = ('name', 'count')).sort_values(by = 'total', ascending=False)
top_chains.head(15)

In [ ]:
# Забыл про признак `"chain"`
top_chains = df[df['chain'] == 1].groupby('name').agg(total = ('name', 'count')).sort_values(by = 'total', ascending=False)
top_chains.head(15)

In [ ]:
# размер графика
fig = plt.subplots(figsize=(20,5), facecolor=(.94, .94, .94))
ax = sns.barplot(x=top_chains[:15].index, y='total', data=top_chains[:15]) 

plt.xticks(rotation=30)

for g in ax.patches:
    ax.annotate(format(g.get_height(), '.1f'),
                   (g.get_x() + g.get_width() / 2., g.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')
plt.xlabel("Название сети", size = 14)
plt.ylabel("Количество заведений", size = 14)
plt.tight_layout()

plt.show()

**Вывод**
Основные популярные сетевые заведения - кафе, кофейни и пиццерии. 

- Какие административные районы Москвы присутствуют в датасете? Отобразим общее количество заведений и количество заведений каждой категории по районам. 


In [ ]:
districts = df.groupby(['district', 'category']).agg(категория=('name', 'count')).unstack()
districts

In [ ]:
ax = districts.sort_values(by=('категория','кофейня')).plot.barh(stacked=True, figsize=(13, 5))
# setting label of y-axis
plt.ylabel("районы")
 
# setting label of x-axis
plt.xlabel("количество заведений")
plt.title("Количество заведений по районам")

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(width), 
            horizontalalignment='center', 
            verticalalignment='center',
            color='white',
            fontsize=10)

plt.show()

**Вывод**
Ожидаемо самое большое количество заведений - в ЦАО. Меньше всего - в СЗАО. 

- Визуализируем распределение средних рейтингов по категориям заведений. Сильно ли различаются усреднённые рейтинги в разных типах общепита?


In [ ]:
rating_categories = df.groupby('category').agg(rating=('rating', 'mean')).sort_values(by='rating', ascending=False)
rating_categories

In [ ]:
# размер графика
fig, ax = plt.subplots(figsize=(10,6), facecolor=(.94, .94, .94))

ax.plot(rating_categories.index, rating_categories['rating'])
 
# заголовок
ax.set_title('Распределение оценок заведений по категориям',
             loc ='left', )

ax.set_ylabel("Оценка заведения")
ax.set_xlabel("Категория заведения")

plt.show()

**Вывод**
Самые плохие рейтинги - у заведений быстрого питания и кафе. Лучше всего - у баров и пабов. 

In [ ]:
# размер графика
fig, ax = plt.subplots(figsize=(10,6), facecolor=(.94, .94, .94))

ax.bar(rating_categories.index, rating_categories['rating'])
 
# заголовок
ax.set_title('Распределение оценок заведений по категориям',
             loc ='left', )

ax.set_ylabel("Оценка заведения")
ax.set_xlabel("Категория заведения")
ax.set_ylim(3)

plt.show()

- Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. Границы районов Москвы, которые встречаются в датасете, хранятся в файле `admin_level_geomap.geojson` 

In [ ]:
rating_df = df.groupby('district', as_index=False)['rating'].agg('mean')
rating_df

In [ ]:
# подключаем модуль для работы с JSON-форматом
import json

# читаем файл и сохраняем в переменной
try:
    with open('/datasets/admin_level_geomap.geojson', 'r') as f:
        geo_json = json.load(f)
except:
    with open('../datasets/admin_level_geomap.geojson', 'r') as f:
        geo_json = json.load(f)


In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = geo_json
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

**Вывод**
Наиболее хорошо оценённые заведения - в центре и на северо-западе. Хуже ситуация на юго-востоке.

- Отобразм все заведения датасета на карте с помощью кластеров средствами библиотеки `folium`.


In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df.apply(create_clusters, axis=1)

# выводим карту
m

<div class="alert alert-info">Чем дальше от центра - тем менее кучно расположены заведения. Особенно это заметно в южных направлениях
    
 
</div>


- Найдём топ-15 улиц по количеству заведений. Построим график распределения количества заведений и их категорий по этим улицам. 


In [ ]:
# ищем топ улиц по количеству заведений
top_streets = df.groupby('street').agg(total = ('name', 'count')).sort_values(by = 'total', ascending=False)
mask = top_streets[:15].index

#считаем заведения по категориям только на выбранных улицах
df_filtered = df[df['street'].isin(mask)]

top_streets_categories = df_filtered.pivot_table(index='street', columns='category', values='address', aggfunc='count')
top_streets_categories

In [ ]:
# stacked
ax = top_streets_categories.plot.barh(stacked=True, figsize=(13, 5))
# setting label of y-axis
plt.ylabel("улицы")
 
# setting label of x-axis
plt.xlabel("количество заведений")
plt.title("Количество заведений по улицам")

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(width), 
            horizontalalignment='center', 
            verticalalignment='center',
            color='white',
            fontsize=10)

plt.show()

**Вывод**
Самое большое количество заведений - на проспекте Мира.

Это довольно протяжённый артериальный проспект с большим количеством станций метро и и автобусных маршрутов

- Найдём улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?


In [ ]:
low_streets = df.groupby('street').agg(total = ('name', 'count'))
low_streets = low_streets[low_streets['total'] == 1]


# заведения только на выбранных улицах
df_filtered_low = df[df['street'].isin(low_streets.index)]
df_filtered_low.head()

In [ ]:
# посмотрим на районы и категории заведений
df_filtered_low_stats = df_filtered_low.pivot_table(index='district', columns='category', values='address', aggfunc='count')
df_filtered_low_stats

In [ ]:
# stacked
ax = df_filtered_low_stats.plot.barh(stacked=True, figsize=(13, 5))
# setting label of y-axis
plt.ylabel("районы")
 
# setting label of x-axis
plt.xlabel("количество заведений")
plt.title("Количество заведений по районам")

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(width), 
            horizontalalignment='center', 
            verticalalignment='center',
            color='white',
            fontsize=10)

plt.show()

In [ ]:
low_districts = df_filtered_low.groupby(['district']).agg(total=('name', 'count'), rating=('rating', 'mean'), middle_avg_bill=('middle_avg_bill', 'mean'), chain=('chain', 'sum'), night=('is_24/7', 'sum'))
low_districts

- Значения средних чеков заведений хранятся в столбце `middle_avg_bill`. Эти числа показывают примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном. Посчитаем медиану этого столбца для каждого района. Используем это значение в качестве ценового индикатора района. Построим фоновую картограмму (хороплет) с полученными значениями для каждого района. 


In [ ]:
df.head()

In [ ]:
bill_districts = df.groupby(['district']).agg(middle_avg_bill=('middle_avg_bill', 'median'))
bill_districts

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = geo_json
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=bill_districts,
    columns=[bill_districts.index, 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианная стоимость заказа в рублях',
).add_to(m)

# выводим карту
m

**Вывод**
В Западном АО средний чек такой же как и в центре, уменьшается чек В северо-восточном, южном и юго-восточном округах

### Промежуточные итоги
- Больше всего в данных представлены кафе, рестораны и кофейни. Кофейни на третьем месте по количеству. Это означает высокую конкуренцию и необходимость грамотно подходить к выбору места для открытия.
- Большинство заведений имеют менее 300 посадочных мест. Большая часть заведений укладывается в диапазон 50-150 посадочных мест
- В датасете больше несетевых заведений, однако кофейни чаще сетевые, чем несетевые. Также сетевыми чаще являются булочные и пиццерии. 
- Среди популярных сетей есть кофейни - One Price Coffee, Cofix, КОФЕПОРТ. 
- Ожидаемо самое большое количество заведений - в ЦАО. Меньше всего - в СЗАО. 
- Самые плохие рейтинги - у заведений быстрого питания и кафе. Лучше всего - у баров и пабов. 
- Наиболее хорошо оценённые заведения - в центре и на северо-западе. Хуже ситуация на юго-востоке.
- В Западном АО средний чек такой же как и в центре, уменьшается чек В северо-восточном, южном и юго-восточном округах

<a id="coffeeshop"></a>
## Детализируем исследование: открытие кофейни

Перед нами следующие вопросы:

- Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?
- Есть ли круглосуточные кофейни?
- Какие у кофеен рейтинги? Как они распределяются по районам?
- На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
# выделим из данных кофейни
coffeeshops = df[df['category'] == 'кофейня'].copy()
coffeeshops.head()

In [ ]:
# информация о датасете
coffeeshops.info()

In [ ]:
coffee_districts = coffeeshops.groupby(['district']).agg(total=('name', 'count'), rating=('rating', 'mean'), middle_coffee_cup=('middle_coffee_cup', 'mean'), chain=('chain', 'sum'), night=('is_24/7', 'sum'))
coffee_districts = coffee_districts.sort_values(by='total', ascending=False)
coffee_districts


In [ ]:
# отсортировать по количеству и показать по районам

# данные
groups = coffee_districts.index
values1 = coffee_districts['total']
values2 = coffee_districts['chain']

# размер графика
fig, ax = plt.subplots(figsize=(20,5), facecolor=(.94, .94, .94))

# Stacked bar chart
ax.bar(groups, values1, label = "Несетевые")
ax.bar(groups, values2, bottom = values1, label = "Сетевые")

# Labels
for bar in ax.patches:
  ax.text(bar.get_x() + bar.get_width() / 2,
          bar.get_height() / 2 + bar.get_y(),
          round(bar.get_height()), ha = 'center',
          color = 'w', weight = 'bold', size = 10)

ax.legend()
ax.set_ylabel('Количество заведений')
ax.set_xlabel('Район')

plt.title('Количество сетевых и несетевых кофеен')

plt.xticks(rotation=30)


plt.show()

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffeeshops.apply(create_clusters, axis=1)

# выводим карту
m

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = geo_json
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=coffeeshops,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

In [ ]:
plt.figure(figsize=(20, 5)) 
ax = sns.boxplot(x='district', y='seats', data=coffeeshops)
plt.ylim(top=400)
plt.xticks(rotation=30)

ax.set_ylabel('Количество посадочных мест')
ax.set_xlabel('Район')

plt.title('Количество посадочных мест в кофейнях')

plt.show()

In [ ]:
coffeeshops.describe()

### Промежуточные итоги
- В датасете 1413 кофеен, больше всего кофеен в центре, на западе, на севере и северо-востоке
- Есть круглосуточные кофейни, но их достаточно мало (больше всего в центре)
- Медианный рейтинг кофеен лучше всего на северо-востоке и востоке. 
- Средняя стоимость чашки в нецентральных, но популярных районах с хорошими рейтингами заведений - 165р